In [43]:
from datetime import datetime
import sys
import time
import sqlite3
import pandas as pd
from gensim.parsing.preprocessing import strip_multiple_whitespaces, preprocess_string, remove_stopwords, strip_tags, strip_punctuation 
import gensim
import re
from tika import parser
import spacy
import string
import nltk

In [ ]:
def initialize_db(db_name):
    connection = sqlite3.connect(db_name)
    connection.row_factory = sqlite3.Row

    cur = connection.cursor()
    cur.execute("SELECT * FROM Jobs")

    return connection, cur

def parse_job(row):
    return {
        'organization' : row['organization'],
        'job_title' : row['job_title'],
        'number_of_openings' : row['number_of_openings'],
        'address_line1' : row['address_line1'],
        'city' : row['city'],
        'province_state' : row['province_state'],
        'postal_code' : row['postal_code'],
        'job_summary' : row['job_summary'],
        'job_responsibilities' : row['job_responsibilities'],
        'required_skills' : row['required_skills'],
        'targeted_disciplines' : row['targeted_disciplines'].replace('\r', '').replace('\t', ' ').replace('\n', '').replace('Targeted Clusters', '')
    }

In [4]:
connection, cur = initialize_db("data/fall2017/jobs.db")

OperationalError: unable to open database file

In [ ]:
count = 0
jobs = []
while True:
    count += 1
    row = cur.fetchone()

    if row is None:
        print("========================== All records processed ============================")
        break

    job = parse_job(row)
    
    jobs.append(job)
    
print("Number of rows loaded: ", count)
connection.close()

In [ ]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(jobs, orient="columns")

In [66]:
df.to_parquet('data/data.parquet', compression="gzip", engine='fastparquet')

NameError: name 'df' is not defined

In [ ]:
#load
data = pd.read_parquet("data.parquet", engine="fastparquet")

In [ ]:
print(data["organization"].values[971])
print(data["job_title"].values[971])
print(data["required_skills"].values[971])
print(len(data))

In [ ]:
def get_cat(row):
    if ("ENG - Software Engineering" in row["targeted_disciplines"]) and ("MATH - Computer Science" in row["targeted_disciplines"]):
        return True
    else:
        return False

In [ ]:
def merge_duplicates(df):
    df = df.drop_duplicates(subset=['organization', 'job_title'], keep="first") 
    df = df.drop_duplicates(subset=['job_responsibilities', 'job_summary', 'required_skills'], keep="first")
    return df

In [ ]:
#Clean data
data["job_responsibilities"] = data["job_responsibilities"].map(lambda x: clean(x))
data["job_summary"] = data["job_summary"].map(lambda x: clean(x))
data["required_skills"] = data["required_skills"].map(lambda x: clean(x))
data = merge_duplicates(data)

In [ ]:
m = data.apply(get_cat, axis=1)
data = data[m]

In [ ]:
data.head()

In [67]:
#create corpus
def preprocess(x):
    x = re.sub(r"\S*@\S*\s?", "", x) #remove emails
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x) #remove website and urls
    
    x = gensim.utils.simple_preprocess(x)
    x = " ".join(x)
    
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, remove_stopwords]
    x = preprocess_string(x, CUSTOM_FILTERS)
    return x
    
def read_corpus():
    for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
        doc = doc1 + " " + doc2 + " " + doc3
        yield gensim.models.doc2vec.TaggedDocument(preprocess(doc), [i]) #formerly gensim.utils.simple_preprocess


In [68]:
train_corpus = list(read_corpus())

In [69]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=75)

In [70]:
model.build_vocab(train_corpus)

In [71]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 32.8 s


In [72]:
#parse resume
raw = parser.from_file('ResumeKevinShen.pdf')
print(raw['content'])
























 

Kevin Shen 
+1 647-994-7088 | kshen3778@gmail.com |​ ​linkedin.com/in/kevinkwshen​ ​|​ ​github.com/kshen3778 

 

Education  

University of Waterloo​, Bachelor of Computer Science Co-op                      2018 -  2023 (expected) 

The Knowledge Society  February 2017 - May 2017 

 

Professional Experience 

Kinaxis     April 2018 - August 2018 
Machine Learning Intern 

● Created and presented a patent-pending machine learning prototype to the CEO and senior executives. 
● Trained and evaluated time series forecasting models such as DeepAR and LightGBM on customer data. 
● Mentored interns on projects that involved sentiment analysis, anomaly detection, and data preprocessing. 

 
Omicron       October 2017 - Present 
Co-founder, Lead Developer  

● Created fully functional platform to collect hackathon data for analytics (launched at a ​150​ people hackathon). 
● Secured over​ $17k+ ​worth of sponsorship for over ​8​ events (​1200+​ attendees combined) si

In [73]:
preprocess("● Created and presented a patent-pending machine learning prototype to the CEO and senior executives. ● Trained and evaluated time series forecasting models such as DeepAR and LightGBM on customer data. ● Mentored interns on projects that involved sentiment analysis, anomaly detection, and data preprocessing." )

['created',
 'presented',
 'patent',
 'pending',
 'machine',
 'learning',
 'prototype',
 'ceo',
 'senior',
 'executives',
 'trained',
 'evaluated',
 'time',
 'series',
 'forecasting',
 'models',
 'deepar',
 'lightgbm',
 'customer',
 'data',
 'mentored',
 'interns',
 'projects',
 'involved',
 'sentiment',
 'analysis',
 'anomaly',
 'detection',
 'data',
 'preprocessing']

In [74]:
nlp = spacy.load('en_core_web_sm')

In [75]:
#Used for Basic Keywords
def clean(x):
    newstr = strip_multiple_whitespaces(x)
    newstr = newstr.strip(u'\u200b')
    newstr = newstr.strip(u'\u200b.')
    newstr = newstr.strip(u'\u200b!')
    newstr = newstr.strip(u'\u200b?')
    #newstr = re.sub(r'[^\w]', "", newstr)
    return newstr


In [76]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kshen\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [105]:
category = "mobile"

part_resume = "Worked on proof-of-concept projects to demonstrate ML feasibility in key supply chain areas. \
Pre-processed time series data, trained forecasting models, and evaluated predictions. \
Mentored other interns on data science libraries, Jupyter, natural language processing, and ML best practices."
# resume = "Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. \
# Created fully functional platform to collect hackathon data for analytics. \
# Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan "
# resume = "Experimented with state of the art NLP techniques to detect type of toxicity in online comments. \
# Experimented with LSTMs, word embeddings (Glove, FastText), and techniques such as sequence bucketing. \
# Fine-tuned and ensembled Bidirectional Encoder Representations from Transformers (BERT). "
# Integrated an “auto-swipe” feature into Tinder based on real-time brain EEG data collected by the Muse. \
# Tested on participants at Hack The 6ix 2017 and achieved a 75% accuracy. \
# Knowledgeable in: Git, Docker, Python (Pandas, Numpy, Scikit-Learn), Keras, Pytorch, Java, Javascript, VueJS, Firebase \
# "



#tagged = nlp(clean(raw["content"])) #THIS IS NOT GOOD, FIND SOME OTHER WAY TO GET KEYWORDS
tagged = nlp(clean(part_resume))
keywords = []

categories = dict()
categories["data science"] = [" data science", " machine learning", " artificial intelligence", " data engineering", " AI", " ML"]
categories["mobile"] = [" Android", " iOS", " IOS", " mobile development", " mobile apps", " apps", " application", " phones", " app development", " mobile applications", " application development"]
categories["web development"] = ["web", " web development", " web applications", " front end", " front-end", " back-end", " back end", " web design", "UI", "UX"]


for token in tagged:
    if token.pos_ == "PROPN":
        keywords.append(token.text)

keywords += categories[category]
keywords = list(set(keywords))

blacklist_words = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
def clean_keywords(words, blacklist):
    cleaned = []
    for word in words:
        print(word)
        regex = re.compile('[@_!#$%^&*\"\'()<>?/\|}{~:]')
        if regex.search(word) == None and len(word) > 2 and (word not in blacklist):
            cleaned.append(clean(word))
    return cleaned
keywords = clean_keywords(keywords, blacklist_words)
    
print(keywords)

#resume_processed = preprocess(raw["content"])
resume_processed = preprocess(part_resume)
print(resume_processed)

 application
Jupyter
 phones
 app development
 IOS
ML
 iOS
 Android
 mobile development
 mobile apps
 apps
 mobile applications
 application development
[' application', 'Jupyter', ' phones', ' app development', ' IOS', ' iOS', ' Android', ' mobile development', ' mobile apps', ' apps', ' mobile applications', ' application development']
['worked', 'proof', 'concept', 'projects', 'demonstrate', 'ml', 'feasibility', 'key', 'supply', 'chain', 'areas', 'pre', 'processed', 'time', 'series', 'data', 'trained', 'forecasting', 'models', 'evaluated', 'predictions', 'mentored', 'interns', 'data', 'science', 'libraries', 'jupyter', 'natural', 'language', 'processing', 'ml', 'best', 'practices']


In [106]:
#Get keyword match score
def score(keywords, job):
    score = 0
    for word in keywords:
        if word in job:
            score += 1
    return score/len(keywords)

In [107]:
inferred_vector = model.infer_vector(resume_processed)
sims = model.docvecs.most_similar([inferred_vector], topn=len(data))

In [109]:
cosine_ranking = dict(sims)

{61: 0.47913044691085815,
 405: 0.39055514335632324,
 439: 0.3751871585845947,
 232: 0.36886975169181824,
 335: 0.3638204336166382,
 539: 0.3492811322212219,
 455: 0.348905086517334,
 572: 0.348588228225708,
 272: 0.34183698892593384,
 399: 0.32342684268951416,
 864: 0.3231435716152191,
 999: 0.32189804315567017,
 199: 0.3188352584838867,
 730: 0.31008195877075195,
 991: 0.3093423843383789,
 1033: 0.308707058429718,
 25: 0.3077464997768402,
 867: 0.3074498176574707,
 1012: 0.30655092000961304,
 761: 0.30454981327056885,
 893: 0.3023066222667694,
 683: 0.3007292151451111,
 876: 0.30017346143722534,
 992: 0.29761508107185364,
 497: 0.2970072627067566,
 829: 0.29462873935699463,
 213: 0.2932274341583252,
 282: 0.29279425740242004,
 77: 0.29258716106414795,
 1030: 0.2915922999382019,
 89: 0.29136067628860474,
 720: 0.29045265913009644,
 379: 0.2875080108642578,
 601: 0.28552883863449097,
 289: 0.2847368121147156,
 127: 0.2836066782474518,
 91: 0.2826704978942871,
 181: 0.28218376636505127,

In [110]:
#get keyword + cosine score
start = time.time()
scores = []
cosine_weight = 0.5
keyword_weight = 0.5
for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
    doc = doc1 + " " + doc2 + " " + doc3
    keyword_score = score(keywords, doc)
    cosine_score = cosine_ranking[i]
    final_score = cosine_weight * cosine_score + keyword_weight * keyword_score
    scores.append((i, final_score))
print(time.time() - start)

0.055544376373291016


In [111]:
sorted_scores = sorted(scores,key=lambda x: x[1], reverse=True)

In [112]:
sorted_scores

[(431, 0.4085276474555333),
 (149, 0.35531219840049744),
 (62, 0.32820664842923486),
 (236, 0.3135315080483755),
 (249, 0.2795657912890116),
 (85, 0.2784518202145895),
 (718, 0.2764004568258921),
 (891, 0.27614518006642663),
 (289, 0.2673684060573578),
 (938, 0.26107873022556305),
 (498, 0.25849130252997077),
 (455, 0.2577858765920003),
 (466, 0.25769665837287903),
 (1110, 0.25685685624678933),
 (272, 0.25425182779630023),
 (287, 0.25346698860327405),
 (580, 0.25315598150094354),
 (1258, 0.2471933811903),
 (1214, 0.2470070719718933),
 (540, 0.2466098815202713),
 (195, 0.24655533333619437),
 (1075, 0.24649625271558762),
 (82, 0.24419634292523065),
 (559, 0.2435232549905777),
 (1097, 0.24160576860109967),
 (61, 0.23956522345542908),
 (219, 0.23888273040453592),
 (405, 0.23694423834482828),
 (171, 0.23674816886583963),
 (1208, 0.23673270580669245),
 (1085, 0.2366068239013354),
 (241, 0.23344576358795166),
 (436, 0.23300963453948498),
 (1197, 0.2327790049215158),
 (213, 0.2299470504124959)

In [113]:
for job in sorted_scores[:50]:
    X = job[0]
    print("ID: ", X)
    print(data["job_title"].values[X], ", ", data["organization"].values[X])
    print()
    print(data["job_responsibilities"].values[X])
    print()
    print(data["job_summary"].values[X])
    print()
    print(data["required_skills"].values[X])
    print("================================")
    print()

ID:  431
Mobile Developer ,  Clearbridge Mobile Inc

Job Description: We are looking for an individual that wants to work in a dynamic working environment to assist our Mobile Development team. If you are a self-starter, love learning about new technologies, and believe mobile is changing the way we work and live, then Clearbridge is the place for you. We provide an excellent learning environment for anyone who has a passion for technology, and you will be under the guidance of our senior developers where there are countless opportunities to learn and grow

Looking for an opportunity to work at one of Canada's top app development companies? Clearbridge Mobile is an award-winning mobile application development studio, and we're looking for a hardworking, enthusiastic co-op student that can join our team of dedicated individuals. We develop custom mobile apps for top tier companies. Whether iOS or Android development, Connected TV, wearables, IoT or beacons, we craft powerful mobile appl

AdHawk Microsystems is changing the way that humans interact with computers by introducing microsystems that can detect eye movements and micro-gestures with unprecedented resolution, speed, and power efficiency. Our eye tracking sensors will transform the way people use Virtual and Augmented Reality (VR/AR) technology (the next major computing platform) through foveated rendering, intuitive and seamless UI design, and enhanced immersion. In addition, our micro-gesture inputs to wearable devices will enable effortless interaction through applications like mid-air swipe keyboards for smart watches. We are a fully funded startup with an experienced management team that believes in the ability of co-op candidates to make a significant contribution - in fact, AdHawk's team members have all been UW engineering coop students in the past. We believe in a flexible work environment with an emphasis on results and outcomes--not hours. Our team consists of researchers and engineers of the highest

- Strong knowledge of Android SDK, different versions of Android and how to deal with different screen sizes - Familiarity with RESTful APIs to connect applications to backend services - Contributed to or have published one or more Android apps to the Play Store. - Solid understanding of the full mobile development life cycle. - Proven working experience in software development (preferably with a startup)

ID:  718
Software Developer ,  Kii Mobile Technology

Required Skills/Knowledge Experience or knowledge of AI and Natural Language Processing Knowledge or coursework in AI and Linguistics Cross-platform mobile development Developing hybrid mobile applications, preferably with Apache Cordova, HTML5, JavaScript/ AngularJS and CSS. Familiar with hybrid mobile app framework, such as IONIC, Mobile Angular UI Developing client-server applications Web service integration (SOAP, REST, JSON, XML) Version control (e.g. SVN, GIT) Student 2nd, 3rd or 4th year BSc / computer programming related d

What you will learn: Prototyping, coding, testing, and improving high-performance software. Collaborating with product managers, UX designers and working with the engineering team and being a part of delivering a compelling product and/or platform Current trends and best practices in building web/mobile applications, including performance, accessibility, security and usability

At Vida Health, we are passionate about transforming lives through better health. We live our vision everyday by creating solutions for preventing and reversing chronic conditions resulting in lower health care costs and better lifestyles for our customers. We are a team that loves building solutions to enable great outcomes for our customers. The team is small enough that you can make an immediate impact on our products but also big enough for you to find your niche and focus on a particular area or technology to transform and further your career aspirations. These internships will be at our Mountain View, CA o

Coursera's mission is to transform lives by providing access to the world's best education. Our engineering team uses a modern tech stack, building all of Coursera on top of AWS. Our website uses React and Webpack written all in ES6, we use Swift for iOS and Java for Android and our backend is built in Scala using MySQL and Cassandra as a datastore. We are looking for rising sophomores and juniors for our spring, summer and winter internship programs. We typically have a dozen interns each program. The programs are highly competitive, so apply as soon as you know you are available. The context team is looking for their next intern! Coursera offers opportunities for learners everywhere, including those who are part of organizations (companies, NGOs, refugee centers). The Context team is building out an experience for such learners (and corresponding organization admins) to up-skill themselves through free access to a number of Coursera courses. This relatively new product, with a massiv

...

Job Qualifications: Programs of Interest: Software Engineering, Computer Science, Computer Engineering, System Design Engineering, Electrical Engineering. Experience with the following is considered as asset: JavaScript Node.js Ember.js HTML5 CSS XML PouchDB Apache CouchDB Apache Cordova Git/Github Android iOS MATLAB Adobe Photoshop UI/UX design The successful applicant should be able to work independently and in a group setting with minimal supervision. This project will involve considerable independent work and the students will be encouraged to contribute his/her own ideas to both subject matter experts and contractors

ID:  1161
Full Stack Developer ,  Accedo

Primary Responsibilities: Implement new functionality in AngularJs, HTML5/JavaScript/TypeScript and CSS Assess and solve existing issues in the current AngularJs application Implement new functionality in the back-end web services built in Node.js and TypeScript Assess and solve existing issues in the various back-end web ser

Compass is a technology-driven real estate platform that is revolutionizing traditional brokerage. We're bringing together some of the world's brightest professionals across real estate, technology, business and creative to simplify and improve the real estate process. We were founded in 2012 by Ori Allon (who started and sold companies to Google & Twitter) and Robert Reffkin (former Chief of Staff to the President & COO of Goldman Sachs) and have raised $210MM from leading investors, including Wellington, IVP, Founders Fund and Thrive Capital. The Company is headquartered in New York City with offices nationwide. The Software Engineering team is located in New York City. You are an up and coming developer who wants to get in at the scale up stage of NYC's fastest growing startup. You want to grow and learn with a world class engineering team of 40 engineers (from firms like Google, Facebook, Apple, Amazon and Twitter). You love to share your knowledge, new ideas and input with others.

All NCR Software Developer/Test personnel have the following characteristics: Outstanding school or work performance Superior analytical and problem solving skills Ability to work independently on diverse problems Team orientation and leadership potential Ability to design, code and correct defects of software modules Excellent oral and written communication skills Diverse interests and a passion for technology Enthusiasm to learn new things The ability to make and meet objectives The ability to work hard while having FUN! Specific technical knowledge and skills required: Java, JEE, Java script Object oriented design Additionally, experience with any of the following technologies and skills would be an asset: JSP and/or JSF AngularJS Mobile development on iOS and/or Android XML programming HTML CSS Hibernate Relational Database Management Systems - SQL programming EJB 3.0 Spring JEE application servers and web servers (IBM WebSphere, GlassFish, Apache Tomcat) Experience developing Web 

Coursera's mission is to transform lives by providing access to the world's best education. Our engineering team uses a modern tech stack, building all of Coursera on top of AWS. Our website uses React and Webpack written all in ES6, we use Swift for iOS and Java for Android and our backend is built in Scala using MySQL and Cassandra as a datastore. We are looking for rising sophomores and juniors for our spring, summer and winter internship programs. We typically have a dozen interns each program. The programs are highly competitive, so apply as soon as you know you are available. The Infrastructure team is looking for their next intern! This team builds the backbone of Coursera's learning platform. They handle all service infrastructure, site operations, frameworks, online data storage, developer tooling and continuous delivery across the full stack. Continuous deployment, server side rendering and graphQL are some of the most recent challenges they are tackling

Pursuing a degree in

Required Qualifications: - Background in machine learning - Professional experience in C++, Java or Python - Superior computer science fundamentals - Strong math chops Preferred Qualifications: - Kaggle competition participant - Experience with deep learning and/or collaborative filtering. - Hands on experience in software systems development and SaaS applications. - Experience with Java, Matlab, Python or Scala

ID:  742
Software Engineer - Intern (Learning Experience) ,  Coursera

As an intern here at Coursera, you will have the mentorship, environment, and the responsibility of deploying your projects to production. We don't throw away intern projects after you leave

Coursera's mission is to transform lives by providing access to the world's best education. Our engineering team uses a modern tech stack, building all of Coursera on top of AWS. Our website uses React and Webpack written all in ES6, we use Swift for iOS and Java for Android and our backend is built in Scala using MySQ